In [6]:
import openai
import pandas as pd
from dotenv import load_dotenv
import os


In [7]:
# Load environment variables from .env file
load_dotenv()

True

In [8]:
# Get the API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')
print(f"API key loaded: {api_key}")

NameError: name 'api_key' is not defined

In [9]:
# Define categories
categories = [
    "Encampment Reporting", "Graffiti", "Pothole", "Animal Complaint", "Weeds & Vegetation", 
    "Neighborhood Issue", "Other", "Snow on Sidewalk", "Abandoned Vehicle", "Illegal Parking", 
    "Damaged/Fallen Tree", "Police: Non-emergency", "Fireworks", "Illegal Dumping", 
    "Shared Micromobility", "No Heat No Water No Electricity", "Missed Trash Pickup"
]

In [10]:
# Define keywords for the initial classification
keywords = {
    "Encampment Reporting": ["encampment", "tent", "homeless"],
    "Graffiti": ["graffiti"],
    "Pothole": ["pothole", "road damage"],
    "Animal Complaint": ["animal", "dog", "cat", "wildlife"],
    "Weeds & Vegetation": ["weeds", "vegetation", "overgrowth", "lawn"],
    "Neighborhood Issue": ["neighborhood", "community", "local issue"],
    "Snow on Sidewalk": ["snow", "sidewalk", "ice"],
    "Abandoned Vehicle": ["abandoned vehicle", "car left", "parked"],
    "Illegal Parking": ["illegal parking", "no parking", "blocked driveway"],
    "Damaged/Fallen Tree": ["fallen tree", "damaged tree", "branch"],
    "Police: Non-emergency": ["police", "non-emergency", "law enforcement"],
    "Fireworks": ["fireworks", "loud noise"],
    "Illegal Dumping": ["illegal dumping", "trash"],
    "Shared Micromobility": ["scooter", "shared bike", "shared micromobility"],
    "No Heat No Water No Electricity": ["no heat", "no water", "no electricity"],
    "Missed Trash Pickup": ["missed trash", "garbage not collected"]
}


In [11]:
# Keyword matching function for classification
def classify_email_initial(email_text):
    email_text_lower = email_text.lower()  # Lowercase for easier matching
    
    # Check for keywords in the email text and assign a category
    for category, words in keywords.items():
        if any(word in email_text_lower for word in words):
            return category
    return "Other"  # Default category if no keywords match


In [18]:
# Function to classify emails using the updated OpenAI API
def classify_email_with_gpt(email_text):
    prompt = f"""
    I have the following email content:
    
    '{email_text}'
    
    Classify this email into one of the following categories:
    1. Encampment Reporting
    2. Graffiti
    3. Pothole
    4. Animal Complaint
    5. Weeds & Vegetation
    6. Neighborhood Issue
    7. Other
    8. Snow on Sidewalk
    9. Abandoned Vehicle
    10. Illegal Parking
    11. Damaged/Fallen Tree
    12. Police: Non-emergency
    13. Fireworks
    14. Illegal Dumping
    15. Shared Micromobility
    16. No Heat No Water No Electricity
    17. Missed Trash Pickup
    
    Reply with the most appropriate category.
    """

    # Use gpt-3.5-turbo or another available model
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that classifies emails."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0.5
    )

    return response['choices'][0]['message']['content'].strip()


In [19]:
# Load the data
file_path = '../Resources/scrubbed_data.csv'
df = pd.read_csv(file_path)
df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforceme...,"Good morning Ron, \n\nThank you for reaching o...",Staff1,NaN,NaN
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to...,Thank you for reaching out to our office. The...,Staff2,NaN,NaN
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,Accident took place at Happy Canyon & Jasmine....,"Good morning TJ,\n\nThank you for reaching out...",Staff1,NaN,NaN
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,Has many concerns about getting a compost cart...,Thank you for reaching out to our office and f...,Staff2,Clara sent another email back with more questi...,Did not respond as they were the same question...
4,Pauline,NaN,pauline@email.com,0,2024-08-21,At any rate my concern is how will HOA condo u...,"Good afternoon Pauline, \n\nThank you for reac...",Staff1,NaN,NaN


In [20]:
# Apply the initial classification
df['initial_category'] = df['constituent_email_1'].apply(lambda email: classify_email_initial(email) if pd.notnull(email) else "Other")


In [21]:
# Apply the GPT-based classification
df['gpt_category'] = df['constituent_email_1'].apply(lambda email: classify_email_with_gpt(email) if pd.notnull(email) else "Other")

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
